In [8]:
#surrogate models
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

data_file = '../all_parts_vectors_drags.csv'
df = TabularDataset(data_file)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=777)

#exclue the first two columns of train data
train_data = train_df.drop(columns=['i', 'name'])
train_data.head()

Loaded data from: ../spvae_vectors_drags.csv | Columns = 131 / 131 | Rows = 439 -> 439


,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_120,dim_121,dim_122,dim_123,dim_124,dim_125,dim_126,dim_127,dim_128,drag
61,-0.996583,0.253377,2.345441,-0.989086,-2.179726,4.450816,-0.333054,-0.833110,0.170525,0.681121,...,1.321793,-0.977492,0.493258,-2.447803,-0.851386,0.003659,-0.704168,0.301570,0.179546,0.375
354,-0.107085,1.786517,1.586284,-1.288352,-1.469519,1.856284,0.022899,0.707441,0.195338,0.934119,...,0.785785,-1.589034,-3.525021,-4.223967,-2.653476,0.012422,0.597222,0.977202,-0.662360,0.374
358,-3.295154,-1.133777,-0.713116,0.946432,0.027521,0.023506,0.565126,-1.224105,-0.519147,0.146645,...,2.272549,-0.996835,5.970934,0.562599,3.063003,-0.003317,1.284646,-0.388674,0.367492,0.435
275,-0.809618,-1.043127,0.594460,1.935878,0.173827,1.941216,-0.689809,-1.535557,1.745006,0.349334,...,0.602183,-0.519529,0.047703,1.788710,-0.448435,-0.003273,-0.998566,-0.302207,0.684875,0.437
18,-0.572277,-0.328782,0.444743,0.411545,-1.628947,3.242414,-0.008619,-0.790840,0.288297,0.432827,...,2.103731,-2.167353,-1.278066,-1.638192,2.981116,0.002460,-0.616031,-0.505367,0.432376,0.367


In [9]:
train_drags = train_df["drag"]
train_index = train_df["i"]
for index, drag in zip(train_index, train_drags):
    print(index, "," ,drag)

61 , 0.375
354 , 0.374
358 , 0.435
275 , 0.437
18 , 0.367
107 , 0.36
57 , 0.456
430 , 0.386
374 , 0.345
179 , 0.345
287 , 0.341
51 , 0.55
395 , 0.48
133 , 0.371
233 , 0.427
210 , 0.478
151 , 0.294
375 , 0.411
24 , 0.538
311 , 0.356
152 , 0.429
369 , 0.478
8 , 0.317
90 , 0.439
47 , 0.418
305 , 0.423
421 , 0.377
253 , 0.418
167 , 0.322
315 , 0.355
345 , 0.489
366 , 0.42
41 , 0.51
359 , 0.453
269 , 0.362
56 , 0.375
193 , 0.408
283 , 0.513
204 , 0.315
224 , 0.333
316 , 0.571
19 , 0.375
150 , 0.33
118 , 0.298
58 , 0.341
223 , 0.359
1 , 0.393
226 , 0.403
124 , 0.339
232 , 0.404
278 , 0.404
342 , 0.344
26 , 0.4
390 , 0.326
379 , 0.417
81 , 0.384
236 , 0.432
100 , 0.374
383 , 0.315
148 , 0.334
3 , 0.337
371 , 0.329
123 , 0.446
231 , 0.348
248 , 0.37
39 , 0.379
120 , 0.439
273 , 0.317
29 , 0.436
378 , 0.371
352 , 0.429
154 , 0.362
407 , 0.347
131 , 0.357
261 , 0.387
262 , 0.402
382 , 0.468
10 , 0.425
88 , 0.402
372 , 0.445
73 , 0.417
234 , 0.516
106 , 0.307
213 , 0.334
364 , 0.356
206 , 0.434
3

In [10]:
label = 'drag'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count    351.000000
mean       0.398513
std        0.060013
min        0.278000
25%        0.353000
50%        0.394000
75%        0.435000
max        0.598000
Name: drag, dtype: float64


In [11]:
%time
import os
import time
start_time = time.time()

save_path = './agModels-all_parts2'  # specifies folder to store trained models
if not os.path.exists(save_path):
    os.makedirs(save_path)

bag_folds = 5 #suggestion range [5, 10], 5
bag_sets = 3 #suggestion range [1, 20], 3
stack_levels = 3 #suggestion range [0, 3], 3
metric = 'root_mean_squared_error' #Regression:mean_absolute_error, mean_squared_error,root_mean_squared_error (default), r2
predictor = TabularPredictor(label=label, path=save_path, eval_metric=metric).fit(train_data, 
                                                                                  presets='best_quality', 
                                                                                  auto_stack="True", 
                                                                                  num_bag_folds=bag_folds, 
                                                                                  num_bag_sets=bag_sets,
                                                                                  num_stack_levels=stack_levels,
                                                                                  verbosity=4)
end_time = time.time()


Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': 'True',
 'num_bag_folds': 0,
 'num_bag_sets': 0,
 'num_stack_levels': 0,
 'verbosity': 4}


CPU times: user 40 µs, sys: 2 µs, total: 42 µs
Wall time: 83.4 µs


Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': 'True',
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': 0,
 'num_bag_sets': 0,
 'num_stack_levels': 0,
 'pseudo_data': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 4}
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=0, num_bag_sets=0
Saving ./agModels-spvae2/learner.pkl
Saving ./agModels-spvae2/predictor.pkl
Beginning AutoGluon training ...
AutoGluon will save models to "./agModels-spvae2/"
AutoGluon Version:  0.7.0
Python Version:     3.10.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #

[1]	valid_set's rmse: 0.0583063
[2]	valid_set's rmse: 0.05801
[3]	valid_set's rmse: 0.0576597
[4]	valid_set's rmse: 0.0571314
[5]	valid_set's rmse: 0.0569612
[6]	valid_set's rmse: 0.0570144
[7]	valid_set's rmse: 0.0567542
[8]	valid_set's rmse: 0.0565622
[9]	valid_set's rmse: 0.0562202
[10]	valid_set's rmse: 0.0558066
[11]	valid_set's rmse: 0.0556623
[12]	valid_set's rmse: 0.0556173
[13]	valid_set's rmse: 0.0553268
[14]	valid_set's rmse: 0.0552455
[15]	valid_set's rmse: 0.0550106
[16]	valid_set's rmse: 0.0550851
[17]	valid_set's rmse: 0.0551156
[18]	valid_set's rmse: 0.0550753
[19]	valid_set's rmse: 0.0549598
[20]	valid_set's rmse: 0.0548997
[21]	valid_set's rmse: 0.0546861
[22]	valid_set's rmse: 0.0546323
[23]	valid_set's rmse: 0.0544171
[24]	valid_set's rmse: 0.0542238
[25]	valid_set's rmse: 0.0541799
[26]	valid_set's rmse: 0.0540439
[27]	valid_set's rmse: 0.0540089
[28]	valid_set's rmse: 0.0539085
[29]	valid_set's rmse: 0.0539308
[30]	valid_set's rmse: 0.0538693
[31]	valid_set's rmse

Saving ./agModels-spvae2/models/LightGBMXT/model.pkl
Saving ./agModels-spvae2/utils/attr/LightGBMXT/y_pred_proba_val.pkl
	-0.0524	 = Validation score   (-root_mean_squared_error)
	0.55s	 = Training   runtime
	0.0s	 = Validation runtime
Saving ./agModels-spvae2/models/trainer.pkl
Fitting model: LightGBM ...
	Dropped 0 of 128 features.
	Fitting LightGBM with 'num_gpus': 0, 'num_cpus': 20
	Fitting 10000 rounds... Hyperparameters: {'learning_rate': 0.05}


...

Saving ./agModels-spvae2/models/XGBoost/model.pkl
Saving ./agModels-spvae2/utils/attr/XGBoost/y_pred_proba_val.pkl
	-0.0549	 = Validation score   (-root_mean_squared_error)
	1.49s	 = Training   runtime
	0.01s	 = Validation runtime
Saving ./agModels-spvae2/models/trainer.pkl
Fitting model: NeuralNetTorch ...
	Dropped 0 of 128 features.
	Fitting NeuralNetTorch with 'num_gpus': 0, 'num_cpus': 20
Tabular Neural Network treats features as the following types:
{
    "continuous": [
        "dim_1",
        "dim_2",
        "dim_3",
        "dim_4",
        "dim_5",
        "dim_6",
        "dim_7",
        "dim_8",
        "dim_9",
        "dim_10",
        "dim_11",
        "dim_12",
        "dim_13",
        "dim_14",
        "dim_15",
        "dim_16",
        "dim_17",
        "dim_18",
        "dim_19",
        "dim_20",
        "dim_21",
        "dim_22",
        "dim_23",
        "dim_24",
        "dim_25",
        "dim_26",
        "dim_27",
        "dim_28",
        "dim_29",
      

[1]	valid_set's rmse: 0.0581641
[2]	valid_set's rmse: 0.057737
[3]	valid_set's rmse: 0.0574427
[4]	valid_set's rmse: 0.0572017
[5]	valid_set's rmse: 0.0570175
[6]	valid_set's rmse: 0.0568971
[7]	valid_set's rmse: 0.0566864
[8]	valid_set's rmse: 0.0565631
[9]	valid_set's rmse: 0.0563735
[10]	valid_set's rmse: 0.0559786
[11]	valid_set's rmse: 0.0555541
[12]	valid_set's rmse: 0.0553156
[13]	valid_set's rmse: 0.0551184
[14]	valid_set's rmse: 0.0547604
[15]	valid_set's rmse: 0.0544081
[16]	valid_set's rmse: 0.0541422
[17]	valid_set's rmse: 0.0538648
[18]	valid_set's rmse: 0.053436
[19]	valid_set's rmse: 0.0534053
[20]	valid_set's rmse: 0.0534636
[21]	valid_set's rmse: 0.0533307
[22]	valid_set's rmse: 0.0531558
[23]	valid_set's rmse: 0.0531692
[24]	valid_set's rmse: 0.0529134
[25]	valid_set's rmse: 0.0529872
[26]	valid_set's rmse: 0.0529593
[27]	valid_set's rmse: 0.0529545
[28]	valid_set's rmse: 0.0528616
[29]	valid_set's rmse: 0.0526544
[30]	valid_set's rmse: 0.0525338
[31]	valid_set's rmse

Saving ./agModels-spvae2/models/LightGBMLarge/model.pkl
Saving ./agModels-spvae2/utils/attr/LightGBMLarge/y_pred_proba_val.pkl
	-0.0514	 = Validation score   (-root_mean_squared_error)
	1.37s	 = Training   runtime
	0.0s	 = Validation runtime
Saving ./agModels-spvae2/models/trainer.pkl
Loading: ./agModels-spvae2/utils/attr/KNeighborsDist/y_pred_proba_val.pkl
Loading: ./agModels-spvae2/utils/attr/ExtraTreesMSE/y_pred_proba_val.pkl
Loading: ./agModels-spvae2/utils/attr/NeuralNetFastAI/y_pred_proba_val.pkl
Loading: ./agModels-spvae2/utils/attr/KNeighborsUnif/y_pred_proba_val.pkl
Loading: ./agModels-spvae2/utils/attr/NeuralNetTorch/y_pred_proba_val.pkl
Loading: ./agModels-spvae2/utils/attr/CatBoost/y_pred_proba_val.pkl
Loading: ./agModels-spvae2/utils/attr/LightGBM/y_pred_proba_val.pkl
Loading: ./agModels-spvae2/utils/attr/LightGBMLarge/y_pred_proba_val.pkl
Loading: ./agModels-spvae2/utils/attr/XGBoost/y_pred_proba_val.pkl
Loading: ./agModels-spvae2/utils/attr/RandomForestMSE/y_pred_proba_v

[331]	valid_set's rmse: 0.0518035
[332]	valid_set's rmse: 0.051803
[333]	valid_set's rmse: 0.0518032
[334]	valid_set's rmse: 0.0518036
[335]	valid_set's rmse: 0.0518089
[336]	valid_set's rmse: 0.0518111
[337]	valid_set's rmse: 0.0518112
[338]	valid_set's rmse: 0.0518065
[339]	valid_set's rmse: 0.0518036
[340]	valid_set's rmse: 0.0518054
[341]	valid_set's rmse: 0.0518051
[342]	valid_set's rmse: 0.0518031
[343]	valid_set's rmse: 0.051804
[344]	valid_set's rmse: 0.0518043
[345]	valid_set's rmse: 0.0518026
[346]	valid_set's rmse: 0.051803
[347]	valid_set's rmse: 0.0518062
[348]	valid_set's rmse: 0.0518071
[349]	valid_set's rmse: 0.0518073


Ensemble size: 14
Ensemble indices: [6, 9, 10, 6, 9, 10, 9, 10, 1, 10, 9, 6, 9, 10]
Ensemble weights: 
[0.         0.07142857 0.         0.         0.         0.
 0.21428571 0.         0.         0.35714286 0.35714286]
Saving ./agModels-spvae2/models/WeightedEnsemble_L2/utils/oof.pkl
Saving ./agModels-spvae2/models/WeightedEnsemble_L2/model.pkl
	-0.0503	 = Validation score   (-root_mean_squared_error)
	0.29s	 = Training   runtime
	0.0s	 = Validation runtime
Saving ./agModels-spvae2/models/trainer.pkl
Saving ./agModels-spvae2/models/trainer.pkl
Saving ./agModels-spvae2/models/trainer.pkl
AutoGluon training complete, total runtime = 18.59s ... Best model: "WeightedEnsemble_L2"
Loading: ./agModels-spvae2/models/trainer.pkl
Saving ./agModels-spvae2/models/trainer.pkl
Saving ./agModels-spvae2/learner.pkl
Saving ./agModels-spvae2/predictor.pkl
Saving ./agModels-spvae2/__version__ with contents "0.7.0"
Saving ./agModels-spvae2/metadata.json
TabularPredictor saved. To load, use: predictor = Ta

In [12]:
print(f"Execution time: {end_time-start_time}")

Execution time: 18.67135715484619


In [13]:
test_data = test_df.drop(columns=['i', 'name'])
# val_data.head()
y_val = test_data[label]
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_119,dim_120,dim_121,dim_122,dim_123,dim_124,dim_125,dim_126,dim_127,dim_128
46,-1.675357,0.774387,-2.380979,-0.728630,-1.412328,1.705713,-0.913276,-0.664502,-0.036643,0.697300,...,2.785418,-0.100383,-1.418045,-5.795298,-0.239288,-3.270577,0.005218,2.296032,-0.461153,1.085652
101,-0.058659,0.094974,-0.148768,-1.709858,0.231730,0.003423,-0.203162,0.288094,-0.420197,-2.209749,...,-2.907042,0.220606,0.879496,-1.250059,-0.911256,2.207671,-0.007707,1.828314,-0.745860,-1.381517
175,0.408514,-0.318794,-0.195711,-1.258788,-0.074261,-3.524118,0.243094,-0.282567,0.518098,1.104262,...,-2.164850,-2.873618,0.817046,1.877096,-1.402977,-0.992778,-0.002622,1.483371,0.380046,0.484612
9,-2.754269,-1.133549,-1.814455,0.036195,-0.159687,2.911953,-0.640023,-0.894014,0.440514,0.164435,...,-2.664808,1.762599,-0.071357,3.233031,-0.915575,2.781991,-0.008800,0.508250,-0.724321,-0.320688
136,-1.054854,0.100548,0.473871,3.140375,-0.449581,2.196428,0.303511,-0.011313,0.375206,-0.866014,...,3.855908,0.273155,-2.094910,-3.537341,1.143077,-2.242801,-0.000889,0.217993,-0.597952,1.817741


In [14]:
# %%capture log_output
# %config InlineBackend.figure_format = 'retina'
# %config Application.log_level = 'DEBUG'
# %config IPCompleter.greedy = True

predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file
y_pred = predictor.predict(test_data_nolab)
for item in y_pred:
    print(item)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_val, y_pred=y_pred, auxiliary_metrics=True)
print(perf)

results = predictor.fit_summary(show_plot=True)
print(results)
print(predictor.leaderboard(test_data, silent=True))

# with open('./output_5040.log', 'w') as f:
#     f.write(log_output.stdout)

Loading: ./agModels-spvae2/predictor.pkl
Loading: ./agModels-spvae2/learner.pkl
Loading: ./agModels-spvae2/models/trainer.pkl
Loading: ./agModels-spvae2/models/ExtraTreesMSE/model.pkl
Loading: ./agModels-spvae2/models/KNeighborsDist/model.pkl
Loading: ./agModels-spvae2/models/LightGBMLarge/model.pkl
Loading: ./agModels-spvae2/models/NeuralNetTorch/model.pkl
Loading: ./agModels-spvae2/models/WeightedEnsemble_L2/model.pkl
Evaluation: root_mean_squared_error on test data: -0.052029913458336574
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -0.052029913458336574,
    "mean_squared_error": -0.002707111894481993,
    "mean_absolute_error": -0.03948913191665302,
    "r2": 0.07940497909157618,
    "pearsonr": 0.3018611278861623,
    "median_absolute_error": -0.03252647447586057
}
Loading: ./agModels-spvae2/models/KNeighborsUnif/model.pkl
Loading: ./agModels-spvae2/models/KN

0.37827032804489136
0.38917893171310425
0.37587660551071167
0.44189774990081787
0.37875816226005554
0.39651840925216675
0.40124717354774475
0.41791316866874695
0.43207114934921265
0.3840968906879425
0.4002808928489685
0.40972644090652466
0.3700295090675354
0.428059458732605
0.3960338234901428
0.4286670386791229
0.3689090609550476
0.4227966070175171
0.40865519642829895
0.36048954725265503
0.43040257692337036
0.41173142194747925
0.3879980742931366
0.38057172298431396
0.4181513786315918
0.3717370629310608
0.41384339332580566
0.38663816452026367
0.3804423213005066
0.3811838626861572
0.42607587575912476
0.3678780794143677
0.40069785714149475
0.4320749044418335
0.38979214429855347
0.37909385561943054
0.37798306345939636
0.3945416808128357
0.38940465450286865
0.38216638565063477
0.43809235095977783
0.3774615526199341
0.4280720055103302
0.4132728576660156
0.4227043390274048
0.42129242420196533
0.36194470524787903
0.3706870377063751
0.4311797320842743
0.4209950566291809
0.3846205472946167
0.355

Loading: ./agModels-spvae2/models/LightGBMXT/model.pkl
Loading: ./agModels-spvae2/models/LightGBM/model.pkl
Loading: ./agModels-spvae2/models/RandomForestMSE/model.pkl
Loading: ./agModels-spvae2/models/CatBoost/model.pkl
Loading: ./agModels-spvae2/models/ExtraTreesMSE/model.pkl
Loading: ./agModels-spvae2/models/NeuralNetFastAI/model.pkl
Loading: ./agModels-spvae2/models/NeuralNetFastAI/model-internals.pkl
Loading: ./agModels-spvae2/models/XGBoost/model.pkl
Loading: ./agModels-spvae2/models/NeuralNetTorch/model.pkl
Loading: ./agModels-spvae2/models/LightGBMLarge/model.pkl
Loading: ./agModels-spvae2/models/WeightedEnsemble_L2/model.pkl
Model scores:
{'KNeighborsUnif': -0.05882093005803317, 'KNeighborsDist': -0.05860647197655768, 'LightGBMXT': -0.05223789616202292, 'LightGBM': -0.05343261811298977, 'RandomForestMSE': -0.051955460753050076, 'CatBoost': -0.05194237651764605, 'ExtraTreesMSE': -0.051587911569212525, 'NeuralNetFastAI': -0.053240822438875304, 'XGBoost': -0.053715460090481025, '

                  model  score_test  score_val  pred_time_test  pred_time_val  \
0         LightGBMLarge   -0.051030  -0.051427        0.010416       0.002570   
1         ExtraTreesMSE   -0.051588  -0.051178        0.153181       0.129661   
2              CatBoost   -0.051942  -0.051571        0.027734       0.018890   
3       RandomForestMSE   -0.051955  -0.052898        0.126400       0.148623   
4   WeightedEnsemble_L2   -0.052030  -0.050320        0.270829       0.414690   
5            LightGBMXT   -0.052238  -0.052405        0.046134       0.002457   
6       NeuralNetFastAI   -0.053241  -0.052606        0.040828       0.010792   
7              LightGBM   -0.053433  -0.053291        0.004919       0.002203   
8               XGBoost   -0.053715  -0.054897        0.026154       0.005693   
9        NeuralNetTorch   -0.056193  -0.052031        0.020953       0.014435   
10       KNeighborsDist   -0.058606  -0.055203        0.075467       0.267614   
11       KNeighborsUnif   -0

In [15]:
best_model = predictor.get_model_best()
print(best_model)

WeightedEnsemble_L2


In [16]:
train_data_pred = predictor.predict(train_data, model=best_model)
test_data_pred = predictor.predict(test_data, model=best_model)

import numpy as np
#save np array y_train_hat to a csv file
np.savetxt('./all_parts_vectors_y_test_hat_sdf.csv', test_data_pred, delimiter=',')
np.savetxt('./all_parts_vectors_y_train_hat_sdf.csv', train_data_pred, delimiter=',')

Loading: ./agModels-spvae2/models/ExtraTreesMSE/model.pkl
Loading: ./agModels-spvae2/models/KNeighborsDist/model.pkl
Loading: ./agModels-spvae2/models/LightGBMLarge/model.pkl
Loading: ./agModels-spvae2/models/NeuralNetTorch/model.pkl
Loading: ./agModels-spvae2/models/WeightedEnsemble_L2/model.pkl
Loading: ./agModels-spvae2/models/ExtraTreesMSE/model.pkl
Loading: ./agModels-spvae2/models/KNeighborsDist/model.pkl
Loading: ./agModels-spvae2/models/LightGBMLarge/model.pkl
Loading: ./agModels-spvae2/models/NeuralNetTorch/model.pkl
Loading: ./agModels-spvae2/models/WeightedEnsemble_L2/model.pkl
